In [24]:
import pandas as pd
from download_versions import RAW_VERSIONS_PATH
import json

In [29]:
# Load the JSON file into a dictionary
title_number = 9
with open(RAW_VERSIONS_PATH / f"title_{title_number}_changes.json", "r") as file:
	data = json.load(file)

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data['content_versions'])
df.shape
# df.head()

(4657, 11)

Questions/ideas:
- how to easily/efficiently get diffs?
- Amendment vs issue date?
- See which titles have most activity per size of current doc
	- substantive
	- removals
	- Could use vectorized names to summarize where most of the changes are happening in a given period. 

In [32]:
df

,date,amendment_date,issue_date,identifier,name,part,substantive,removed,subpart,title,type
0,2016-11-08,2016-11-08,2016-12-20,1.1,§ 1.1 Definitions.,1,True,False,None,9,section
1,2016-11-08,2016-11-08,2016-12-20,2.1,§ 2.1 Requirements and application.,2,True,False,A,9,section
2,2016-11-08,2016-11-08,2016-12-20,2.2,§ 2.2 Acknowledgement of regulations and sta...,2,True,False,A,9,section
3,2016-11-08,2016-11-08,2016-12-20,2.3,§ 2.3 Demonstration of compliance with stand...,2,True,False,A,9,section
4,2016-11-08,2016-11-08,2016-12-20,2.4,§ 2.4 Non-interference with APHIS officials.,2,True,False,A,9,section
...,...,...,...,...,...,...,...,...,...,...,...
4652,2023-09-18,2023-09-18,2025-04-10,381.76,§ 381.76 Post-mortem inspection under Tradit...,381,False,False,K,9,section
4653,2016-12-20,2016-12-20,2025-04-10,381.158,§ 381.158 Poultry dinners (frozen) and pies.,381,False,False,P,9,section
4654,2019-12-27,2019-12-27,2025-04-10,381.196,§ 381.196 Eligibility of foreign countries f...,381,False,False,T,9,section
4655,2016-12-20,2016-12-20,2025-04-10,381.412,§ 381.412 Reference amounts customarily cons...,381,False,False,Y,9,section
